# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [1]:
from itertools import combinations
from heapq import heappush, heappop
import math

n = 2
num_people = 2 * n

def initial_state():
    """ All people and the boat are on the left bank."""
    return (('L',) * num_people, 'L')

def is_goal(state):
    locations, _ = state
    return all(loc == 'R' for loc in locations)

def is_valid(S):
    """Checks if the wife (W) is with her husband (H = W - 1) or not."""
    husbands = {p for p in S if p % 2 == 0}
    for w in S:
        if w % 2 == 1:  # Wife
            h = w - 1  # Her husband
            if h not in husbands and husbands:  # Husband absent, other husbands present
                return False
    return True

def is_valid_state(state):
    """Checks if the state is valid across all locations."""
    locations, _ = state
    L_people = {i for i in range(num_people) if locations[i] == 'L'}
    I_people = {i for i in range(num_people) if locations[i] == 'I'}
    R_people = {i for i in range(num_people) if locations[i] == 'R'}
    return is_valid(L_people) and is_valid(I_people) and is_valid(R_people)

def successors(state):
    """Generates all valid successor states."""
    locations, boat = state
    if boat == 'L':
        possible_moves = [('L', 'I')]
    elif boat == 'I':
        possible_moves = [('I', 'L'), ('I', 'R')]
    else: 
        possible_moves = [('R', 'I')]
    
    for start, dest in possible_moves:
        current_people = [i for i in range(num_people) if locations[i] == start]
        for r in [1, 2]:
            for people in combinations(current_people, min(r, len(current_people))):
                new_locations = list(locations)
                for p in people:
                    new_locations[p] = dest
                new_state = (tuple(new_locations), dest)
                if is_valid_state(new_state):
                    yield new_state

def state_to_string(state):
    """Converts state to readable string for printing."""
    locations, boat = state
    people = [f'H{i//2 + 1}' if i % 2 == 0 else f'W{(i+1)//2}' for i in range(num_people)]
    loc_str = ' '.join(f'{p} {loc}' for p, loc in zip(people, locations))
    return f"{loc_str} Boat: {boat}"


# DFS Solution
def dfs():
    start = initial_state()
    stack = [(start, [])]  # (state, path)
    visited = set()
    while stack:
        state, path = stack.pop()
        if state in visited:
            continue
        visited.add(state)
        if is_goal(state):
            return path + [state]
        for next_state in successors(state):
            if next_state not in visited:
                stack.append((next_state, path + [state]))
    return None

# Backtracking Solution
def backtracking(state=None, path=None, visited=None):
    if state is None:
        state = initial_state()
        path = []
        visited = set()
    
    if state in visited:
        return None
        
    visited.add(state)
    if is_goal(state):
        return path + [state]
    for next_state in successors(state):
        if next_state not in visited:
            result = backtracking(next_state, path + [state], visited.copy())
            if result:
                return result
    return None

# A* Heuristic
def heuristic(state):
    """heuristic: [people not on R + 1] / 2."""
    locations, _ = state
    not_on_R = sum(loc != 'R' for loc in locations)
    return math.ceil(not_on_R / 2)

# A* Solution
def astar():
    start = initial_state()
    frontier = []  # (f, g, state, path)
    heappush(frontier, (heuristic(start), 0, start, []))
    visited = set()
    while frontier:
        f, g, state, path = heappop(frontier)
        if state in visited:
            continue
        visited.add(state)
        if is_goal(state):
            return path + [state]
        for next_state in successors(state):
            if next_state not in visited:
                new_g = g + 1
                h = heuristic(next_state)
                heappush(frontier, (new_g + h, new_g, next_state, path + [state]))
    return None

def print_solution(method_name, solution):
    print(f"\n{method_name} Solution:")
    if solution:
        for i, state in enumerate(solution, 1):
            print(f"Step {i}: {state_to_string(state)}")
        print(f"Total steps: {len(solution) - 1}")
    else:
        print("No solution found.")

print_solution("DFS", dfs())
print_solution("Backtracking", backtracking())
print_solution("A*", astar())


DFS Solution:
Step 1: H1 L W1 L H2 L W2 L Boat: L
Step 2: H1 L W1 L H2 I W2 I Boat: I
Step 3: H1 L W1 L H2 R W2 R Boat: R
Step 4: H1 L W1 L H2 I W2 R Boat: I
Step 5: H1 L W1 L H2 L W2 R Boat: L
Step 6: H1 I W1 L H2 I W2 R Boat: I
Step 7: H1 R W1 L H2 R W2 R Boat: R
Step 8: H1 R W1 L H2 I W2 I Boat: I
Step 9: H1 R W1 L H2 R W2 I Boat: R
Step 10: H1 I W1 L H2 I W2 I Boat: I
Step 11: H1 L W1 L H2 L W2 I Boat: L
Step 12: H1 L W1 I H2 L W2 I Boat: I
Step 13: H1 L W1 R H2 L W2 R Boat: R
Step 14: H1 L W1 R H2 L W2 I Boat: I
Step 15: H1 L W1 R H2 L W2 L Boat: L
Step 16: H1 L W1 R H2 I W2 I Boat: I
Step 17: H1 L W1 R H2 L W2 I Boat: L
Step 18: H1 I W1 R H2 I W2 I Boat: I
Step 19: H1 R W1 R H2 R W2 I Boat: R
Step 20: H1 R W1 R H2 I W2 I Boat: I
Step 21: H1 R W1 R H2 R W2 R Boat: R
Total steps: 20

Backtracking Solution:
Step 1: H1 L W1 L H2 L W2 L Boat: L
Step 2: H1 I W1 I H2 L W2 L Boat: I
Step 3: H1 L W1 I H2 L W2 L Boat: L
Step 4: H1 L W1 I H2 L W2 I Boat: I
Step 5: H1 L W1 L H2 L W2 I Boat: